In [1]:
from lxml import etree
from lxml.etree import tostring, parse
import wikitextparser as wtp
import pandas as pd
import re

In [2]:
text = open("Wikipedia-20201015222842.xml", encoding = "UTF-8").read()
parsed = wtp.parse(text)
in_links = parsed.wikilinks

In [3]:
list_links = list(map(lambda x : x.title, in_links))
list_links

['WP:AWB/GF',
 'WP:AWB/T',
 'Inter Milan',
 'Rosario, Santa Fe',
 'Striker (association football)',
 'Paris Saint-Germain F.C.',
 'UD Vecindario',
 'FC Barcelona',
 'U.C. Sampdoria',
 'U.C. Sampdoria',
 'Inter Milan',
 'Paris Saint-Germain F.C.',
 'Paris Saint-Germain F.C.',
 'Argentina national under-20 football team',
 'Argentina national football team',
 'Association football',
 'Striker (association football)',
 'Ligue 1',
 'Paris Saint-Germain F.C.',
 'Argentina national football team',
 'La Masia',
 'La Liga',
 'FC Barcelona',
 'Serie A',
 'U.C. Sampdoria',
 'Inter Milan',
 'Capocannoniere',
 '2014–15 Serie A',
 '2017–18 Serie A',
 'Luca Toni',
 'Ciro Immobile',
 'UEFA Champions League',
 'List of Inter Milan records and statistics',
 'Argentina national football team',
 '2018 FIFA World Cup',
 'Wanda Nara',
 'Maxi López',
 'Rosario, Santa Fe',
 'Canary Islands',
 'UD Vecindario',
 'Gran Canaria',
 'FC Barcelona',
 'Real Madrid C.F.',
 'Valencia CF',
 'Sevilla FC',
 'RCD Espanyol

In [ ]:
enwiki-20201001-pages-articles-multistream1.xml-p1p41242

In [15]:
text = open("enwiki-latest-abstract20.xml", encoding = "UTF-8").read()

In [7]:
parsed = wtp.parse(text)
in_links = parsed.wikilinks

In [ ]:
parsed.sections

In [36]:
import mwxml

In [ ]:
dump = mwxml.Dump.from_file(text)

In [ ]:

print(dump.site_info.name, dump.site_info.dbname)
Wikipedia enwiki
for page in dump:
for revision in page:
    print(revision.id)

In [30]:
from lxml import etree
t = etree.XML(text)

In [ ]:
(t.text + ''.join(map(etree.tostring, t))).strip()

In [99]:
pa = ["disambiguation/ oheeoc"]
if "disambiguation" in pa[0]:
    print("oui")

oui


# A TESTER

In [2]:
import bz2
import subprocess

In [3]:
data_path = 'enwiki-20201001-pages-articles1.xml-p1p41242.bz2'

In [4]:
lines = []
for i, line in enumerate(bz2.BZ2File(data_path, 'r')):
    lines.append(line)
    if i > 5e5:
        break

In [5]:
import xml.sax

class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            if "disambiguation" not in self._values['text']:
                self._pages.append((self._values['title'], self._values['text']))

In [6]:
# Content handler for Wiki XML
handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

handler._pages

[]

In [7]:
# Object for handling xml
handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

for line in bz2.BZ2File(data_path, 'r'):
    parser.feed(line)

    if len(handler._pages) > 1000:
        break
        
print([x[0] for x in handler._pages])

['AccessibleComputing', 'AfghanistanHistory', 'AfghanistanGeography', 'AfghanistanPeople', 'AfghanistanCommunications', 'AfghanistanTransportations', 'AfghanistanMilitary', 'AfghanistanTransnationalIssues', 'AssistiveTechnology', 'AmoeboidTaxa', 'Autism', 'AlbaniaHistory', 'AlbaniaPeople', 'AsWeMayThink', 'AlbaniaGovernment', 'AlbaniaEconomy', 'Albedo', 'AfroAsiaticLanguages', 'ArtificalLanguages', 'AbacuS', 'AbalonE', 'AbbadideS', 'AbbesS', 'AbbevilleFrance', 'AbbeY', 'AbboT', 'Abbreviations', 'AtlasShrugged', 'ArtificialLanguages', 'AtlasShruggedCharacters', 'AtlasShruggedCompanies', 'AyersMusicPublishingCompany', 'AfricanAmericanPeople', 'AdolfHitler', 'AbeceDarians', 'AbeL', 'AbensbergGermany', 'AberdeenSouthDakota', 'ArthurKoestler', 'AynRand', 'AlexanderTheGreat', 'AnchorageAlaska', 'ArgumentForms', 'ArgumentsForTheExistenceOfGod', 'AnarchY', 'AsciiArt', 'AcademyAwards', 'AcademyAwards/BestPicture', 'AustriaLanguage', 'AcademicElitism', 'AxiomOfChoice', 'AmericanFootball', 'Ameri

In [8]:
import mwparserfromhell 

chosen_page = handler._pages[560]
print(chosen_page[0])

# Create the wiki article
wiki = mwparserfromhell.parse(chosen_page[1])

Ahmed II


In [9]:
wikilinks = [x.title for x in wiki.filter_wikilinks()]
print(f'There are {len(wikilinks)} wikilinks.')
wikilinks[:5]

There are 66 wikilinks.


['Caesar (title)#Ottoman Empire',
 'Custodian of the Two Holy Mosques',
 'Ottoman Caliphate',
 'Suleiman II of the Ottoman Empire',
 'Mustafa II']

In [10]:
tr = wiki.strip_code().strip()
tr

'Ahmed II (  Aḥmed-i  < u > s < /u > ānī)  < span dir= " ltr " > (25 February 1643 or 1 August 1642 < ref > Mantran, Robert, “Aḥmad II”, in Encyclopaedia of Islam, Second Edition, vol.I (1986) p.268 : first date according to Naima, second date to Raşid < /ref >  – 6 February 1695) < /span >  was the Sultan of the Ottoman Empire from 1691 to 1695. \n \n == Early life == \n Ahmed II was born on 25 February 1643 or 1 August 1642, the son of Sultan Ibrahim and Muazzez Sultan. On 21 October 1649, Ahmed along with his brothers Mehmed and Suleiman were circumcised.   \n \n During the reigns of his older brothers, Ahmed was imprisoned in Kafes, and he stayed there almost 43 years. \n thumb|262px|The mausoleum of Ahmed II is located inside the türbe of Suleiman the Magnificent.  < small > (In the above picture, his tomb is seen side by side with Suleiman II and Suleiman the Magnificent). < /small >  \n \n ==Reign== \n  \n During his reign, Sultan Ahmed II devoted most of his attention to the wa

In [11]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext
cleaned_text = cleanhtml(tr)

In [12]:
print(cleaned_text).split("\n")

Ahmed II (  Aḥmed-i   s  ānī)   (25 February 1643 or 1 August 1642  Mantran, Robert, “Aḥmad II”, in Encyclopaedia of Islam, Second Edition, vol.I (1986) p.268 : first date according to Naima, second date to Raşid   – 6 February 1695)   was the Sultan of the Ottoman Empire from 1691 to 1695. 
 
 == Early life == 
 Ahmed II was born on 25 February 1643 or 1 August 1642, the son of Sultan Ibrahim and Muazzez Sultan. On 21 October 1649, Ahmed along with his brothers Mehmed and Suleiman were circumcised.   
 
 During the reigns of his older brothers, Ahmed was imprisoned in Kafes, and he stayed there almost 43 years. 
 thumb|262px|The mausoleum of Ahmed II is located inside the türbe of Suleiman the Magnificent.   (In the above picture, his tomb is seen side by side with Suleiman II and Suleiman the Magnificent).   
 
 ==Reign== 
  
 During his reign, Sultan Ahmed II devoted most of his attention to the wars against the Habsburgs and related foreign policy, governmental and economic issues.

AttributeError: 'NoneType' object has no attribute 'split'